복습복습복습 !!!!!!

# 국민 청원 데이터 분석
* 시계열 분석 필요함.

csv
* answered : 청와대 답변 여부 확인
* 답변 대기 중인 청원이 존재함

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = '../datasets/ML_data/'

In [5]:
raw = pd.read_csv(path+'petition.csv', on_bad_lines='skip', encoding='utf8', index_col=0, parse_dates=['start', 'end'])
petition = raw.copy()
print(petition.shape)
petition.head(2)

(377756, 7)


,start,end,answered,votes,category,title,content
article_id,,,,,,,
21,2017-08-19,2017-11-17,0,9,안전/환경,스텔라 데이지호에 대한 제안입니다.,스텔라 데이지호에 대한 제안입니다.\n3월31일 스텔라 데이지호가 침몰하고 5달째가...
22,2017-08-19,2017-11-17,0,17,기타,비리제보처를 만들어주세요.,현 정부에 국민들이 가장 원하는 것은 부패척결입니다. 우리 사회에 각종 비리들이 ...


In [6]:
petition.info()

<class 'pandas.core.frame.DataFrame'>
Index: 377756 entries, 21 to 492043
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   start     377756 non-null  datetime64[ns]
 1   end       377756 non-null  datetime64[ns]
 2   answered  377756 non-null  int64         
 3   votes     377756 non-null  int64         
 4   category  377756 non-null  object        
 5   title     377756 non-null  object        
 6   content   377755 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 23.1+ MB


In [7]:
petition.columns

Index(['start', 'end', 'answered', 'votes', 'category', 'title', 'content'], dtype='object')

In [8]:
petition.tail(2)

,start,end,answered,votes,category,title,content
article_id,,,,,,,
492042,2019-01-09,2019-02-08,0,4,정치개혁,임종석실장님 수고많으셨습니다.,범죄정권이후 많은 어려움을 갖고 시작한 국민의정부.\n저급한 자칭 보수단체와 한국당...
492043,2019-01-09,2019-02-08,0,1,행정,예천군과 환경부를 규탄합니다. 어불성설인 가축사육 관련 법규를 개정해주세요!,해당 사건이 발생한 곳은 요즘은 매체에서 매일 나오는 곳으로 '군의원의 외유성 해외...


In [10]:
petition.describe(include='all')

,start,end,answered,votes,category,title,content
count,377756,377756,377756.000000,3.777560e+05,377756,377756,377755
unique,NaN,NaN,NaN,NaN,17,330206,358060
top,NaN,NaN,NaN,NaN,정치개혁,이명박 출국금지,이명박 출국금지
freq,NaN,NaN,NaN,NaN,59020,3018,597
mean,2018-05-15 05:35:15.155814912,2018-06-15 04:09:46.359449088,0.000077,1.514069e+02,NaN,NaN,NaN
min,2017-08-19 00:00:00,2017-08-26 00:00:00,0.000000,0.000000e+00,NaN,NaN,NaN
25%,2018-01-22 00:00:00,2018-02-21 00:00:00,0.000000,1.000000e+00,NaN,NaN,NaN
50%,2018-05-19 00:00:00,2018-06-18 00:00:00,0.000000,5.000000e+00,NaN,NaN,NaN
75%,2018-09-05 00:00:00,2018-10-05 00:00:00,0.000000,1.500000e+01,NaN,NaN,NaN
max,2019-01-09 00:00:00,2019-02-08 00:00:00,1.000000,1.192049e+06,NaN,NaN,NaN


In [11]:
# 결측치 확인
petition.isnull().sum()

start       0
end         0
answered    0
votes       0
category    0
title       0
content     1
dtype: int64

## 기본적인 전처리

In [21]:
# 20만건 이상의 투표가 있는 청원인지 확인하기
petition['answer'] = (petition.votes >= 200000) == 1
petition.head(2)

,start,end,answered,votes,category,title,content,duration,answer
article_id,,,,,,,,,
21,2017-08-19,2017-11-17,0,9,안전/환경,스텔라 데이지호에 대한 제안입니다.,스텔라 데이지호에 대한 제안입니다.\n3월31일 스텔라 데이지호가 침몰하고 5달째가...,90 days,False
22,2017-08-19,2017-11-17,0,17,기타,비리제보처를 만들어주세요.,현 정부에 국민들이 가장 원하는 것은 부패척결입니다. 우리 사회에 각종 비리들이 ...,90 days,False


시계열 데이터
* 같은 시기의 과거 데이터와 비교를 많이 한다.

In [17]:
# 청원 기간에 대한 컬럼 생성
petition['duration'] = petition['end'] - petition['start']
petition.head(2)

,start,end,answered,votes,category,title,content,anwser,duration
article_id,,,,,,,,,
21,2017-08-19,2017-11-17,0,9,안전/환경,스텔라 데이지호에 대한 제안입니다.,스텔라 데이지호에 대한 제안입니다.\n3월31일 스텔라 데이지호가 침몰하고 5달째가...,False,90 days
22,2017-08-19,2017-11-17,0,17,기타,비리제보처를 만들어주세요.,현 정부에 국민들이 가장 원하는 것은 부패척결입니다. 우리 사회에 각종 비리들이 ...,False,90 days


In [18]:
# 청원 기간이 작은 순으로 확인기기
petition.sort_values(by='duration', ascending=True)

,start,end,answered,votes,category,title,content,anwser,duration
article_id,,,,,,,,,
13577,2017-09-12,2017-09-19,0,20,육아/교육,사립유치원의 집단휴업을 반대합니다,국공립 유치원도 없고 국공립어린이집 들어가기도 힘든 실정에 사립유치원은 부모들의 어...,False,7 days
4213,2017-09-05,2017-09-12,0,1,인권/성평등,청소년 보호법이 아니라 소년법입니다...,소년법 폐지 청원합니다,False,7 days
4214,2017-09-05,2017-09-12,0,0,정치개혁,소년법 강화,이번 부산여중생 사건을보고 느낍니다.\n소년법 폐지해주셨으면 좋겠습니다.\n아님 강...,False,7 days
12522,2017-09-08,2017-09-15,0,7,정치개혁,호소문,호 소 문\n새 대통령님 안녕하십니까? 저는 금오공과대학교 인라인 동아리 회장 차형...,False,7 days
4218,2017-09-05,2017-09-12,0,0,미래,나라의미래를위해 청소년보호법 폐지해주시길 간절히바랍니다,청소년보호법은 청소년을 위한법이아니라 청소년을 위협하는법이 됬습니다 부디 바로선 법...,False,7 days
...,...,...,...,...,...,...,...,...,...
8224,2017-09-05,2017-12-04,0,0,안전/환경,소년법 폐지,소년법을강화시켜주십시오!,False,90 days
8226,2017-09-05,2017-12-04,0,0,기타,청소년보호법 폐지해주세요,청소년보호법 폐지되야된다고 생각합니다 폐지되길 바래요,False,90 days
8227,2017-09-05,2017-12-04,0,0,정치개혁,소년법폐지,제발 소년법 폐지해주세요. 이런 끔찍한 일이 더이상 없어야합니다.,False,90 days


In [19]:
# 청원 기간별 건수 확인
petition.duration.value_counts()

duration
30 days    366556
90 days      6754
7 days       2436
15 days      1204
60 days       806
Name: count, dtype: int64

보통 청원하면 한달 정도 걸림.

In [22]:
# 청원 기간이 90일일 때, 답변 대상 건수
petition[(petition.duration == '90 days') & (petition.answer == 1)]

,start,end,answered,votes,category,title,content,duration,answer
article_id,,,,,,,,,
10949,2017-09-06,2017-12-05,1,615354,미래,조두순 출소반대,제발 조두순 재심다시해서 무기징역으로 해야됩니다!!!,90 days,True


In [23]:
# 청원 기간이 60일일 때, 답변 대상 건수
petition[(petition.duration == '60 days') & (petition.answer == 1)]

,start,end,answered,votes,category,title,content,duration,answer
article_id,,,,,,,,,
1785,2017-09-03,2017-11-02,1,296330,인권/성평등,청소년이란 이유로 보호법을 악용하는 잔인무도한 청소년들이 늘어나고있습니다. 반드시 ...,안녕하십니까. 청소년보호법이란 명목하에 나쁜짓을 일삼는 청소년들이 너무나 많아지고 ...,60 days,True


In [24]:
# 청원 기간이 30일일 때, 답변 대상 건수
petition[(petition.duration == '30 days') & (petition.answer == 1)]

,start,end,answered,votes,category,title,content,duration,answer
article_id,,,,,,,,,
18278,2017-09-30,2017-10-30,1,235372,인권/성평등,낙태죄 폐지와 자연유산 유도약(미프진) 합법화 및 도입을 부탁드립니다.,안녕하세요. 존경하는 대통령님 의원님\n낙태죄 폐지를 청원합니다.\n현재 대한민국은...,30 days,True
26024,2017-11-04,2017-12-04,1,216774,안전/환경,'주취감형(술을 먹으면 형벌 감형)' 폐지를 건의(청원)합니다.,"'주취감형'이란, 술을 먹고 범행을 저지를 때, 심신미약 (이성이 없고 우발적)이라...",30 days,True
45581,2017-11-17,2017-12-17,1,281985,보건복지,"권역외상센터 (이국종 교수님) 추가적, 제도적, 환경적, 인력 지원","청원사항)\n중증외상분야의 추가적, 제도적, 환경적, 인력 지원 방안마련\n현 의료...",30 days,True
49083,2017-11-24,2017-12-24,1,211064,기타,"전안법. 18살, 미성년자에게 정부가 직접 찍어주는 범죄자 낙인!","3년 이하의 징역, 또는3천만원 이하의 벌금. 혹은 500만원 이하의 과태료.\n설...",30 days,True
57132,2017-12-03,2018-01-02,1,255554,성장동력,전안법 합리적으로 개정 또는 폐지해주세요.(소상공인 소비자 모두 죽는법안입니다.),2017년 1월 28일부터 시행되는 법령으로 국가기술표준원에서 관리하는 전기제품에만...,30 days,True
...,...,...,...,...,...,...,...,...,...
454045,2018-11-27,2018-12-27,0,214251,반려동물,도살장에서 망치로 머리를 맞던 개가 잠시 튀어나가 옆에있는 자신의 새끼에게 젖물리며...,아래의 링크를 봐주세요\n사람과 똑같이 자신의 자식을 귀히여기고 사람과 똑같이 죽는...,30 days,True
462340,2018-12-06,2019-01-05,0,221317,육아/교육,짐승보다 못한 위탁모 에게 굶기고 맞아죽은 15개월된 저희딸 얘기좀 들어주세요....,짐승 같은 위탁모(김모씨) 에게 맞아 죽은 15개월 된 저희 딸 서원이가 억울하게 ...,30 days,True
462623,2018-12-07,2019-01-06,0,243938,정치개혁,국회의원 내년 연봉 2000만원 인상 추진..최저임금 인상률보다 높은 14%.. 셀...,국회의원 내년 연봉 2000만원 인상 추진..최저임금 인상률보다 높은 14%로 인상...,30 days,True


In [29]:
# 청원 기간이 30일일 때, 답변 대상 건수
petition[(petition.duration == '7 days') & (petition.answer != 1)].start.value_counts()

start
2017-09-05    942
2017-09-06    335
2017-09-04    331
2017-09-07    129
2017-09-08     94
2017-09-03     53
2017-09-01     52
2017-09-09     43
2017-09-12     41
2017-08-30     40
2017-09-02     39
2017-09-13     37
2017-08-31     37
2017-09-10     32
2017-08-29     27
2017-08-28     24
2017-08-23     22
2017-08-21     21
2017-08-25     20
2017-09-11     20
2017-08-26     19
2017-08-20     18
2017-08-24     18
2017-08-22     15
2017-08-27     11
2017-08-19      9
2017-09-14      7
Name: count, dtype: int64

In [33]:
# category별 청원 수 확인
category_petition = petition.category.value_counts().reset_index()
category_petition.head(2)

,category,count
0,정치개혁,59020
1,기타,46449


In [35]:
# 청원이 얼마동안 집계?
petition.start.value_counts().reset_index()

,start,count
0,2017-11-11,9623
1,2017-09-05,5952
2,2018-01-11,3368
3,2018-02-06,2631
4,2017-11-09,2487
...,...,...
504,2017-09-15,74
505,2017-08-22,69
506,2017-08-27,49
507,2017-08-19,39


In [ ]:
# timedelta : 시간의 차이를 구하는 ~


In [36]:
# pivot_table로 투표를 가장 많이 받은 날은 어디인지 확인하기
petition_start = pd.pivot_table(data=petition, index=['start'], aggfunc=agg_func)
petition_start.head(2)

NameError: name 'agg_func' is not defined

In [37]:
# 위에서 만든 피벗 테이블을  투표수로 내림차순 정렬하기

# 투표수가 20만건 이상인 것만 확인기기

In [ ]:
# 청원을 많이 받은 날 vs. 투표를 많이 받은 날
# merge하기 - 'start' 기준 / 왼쪽


In [ ]:
# 가장 핫한 날 확인하기 - 투표를 가장 많이 한 날
# nlargest 확인

In [ ]:
# 가장 핫한 날 확인하기 - 청원을 가장 많이 한 날
# nlargest 확인

In [ ]:
# 답변 대상인 청원 확인하기
# vote가 20만건 이상인 데이터만 추출해서 따로 DF로 저장하기

In [ ]:
# 답변 대상인 청원 중 vote가 많은 것은 무엇일까?
# sort_values, 내림차순 확인하기


In [ ]:
# plotnine 다운 받기 - R 코드 사용한다.
# 한글폰트 사용 다운 받기
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'
font_family = 'NanumGothic'
font_family

In [ ]:
# 시각화하기

(ggplot(petition) 
 + aes('category') 
 + geom_bar(fill='green')
 + theme(text=element_text(family=font_family),
         axis_text_x=element_text(rotation=60)))

In [ ]:
# category별 투표수
(ggplot(petition)
 + aes(x='category', y='votes')
 + geom_col(fill='skyblue')
 + ggtitle('카테고리별 투표수')
 + coord_flip()
 + theme(text=element_text(family=font_family)))

인권 성평등이 많이 나오는 것을 알 수 있었음. 따라서 이것에 대한 추가 분석이 들어가야 한다.

In [39]:
# 투표를 가장 많이 받은 카테고리 : 인권 / 성평등
# new DF로 저장
human_rights = petition.loc[(petition.category == '인권/성평등')]

# votes로 내림차순 정렬하기
human_rights.sort_values(by='votes', ascending=False)[:5]

,start,end,answered,votes,category,title,content,duration,answer
article_id,,,,,,,,,
230552,2018-05-11,2018-06-10,1,419006,인권/성평등,여성도 대한민국 국민입니다. 성별 관계없는 국가의 보호를 요청합니다.,최근 홍대 누드크로키 모델의 불법촬영 사건이 있었습니다.\n사건은 굉장히 빠르게 처...,30 days,True
426834,2018-10-31,2018-11-30,0,416093,인권/성평등,"132cm, 31kg의 왜소한 50대 여성이 180cm가 넘는 건장한 20세 남성에...",http://m.news1.kr/articles/?3464603\n관련 기사입니다....,30 days,True
437737,2018-11-09,2018-12-09,0,397079,인권/성평등,억울하게 떠나신 저의 아버지의 원한을 풀어주세요,46일동안 눈 한번 뜨시지 못한 채 11월 6일 새벽 4시에 저의 아버지가 억울하게...,30 days,True
442093,2018-11-14,2018-12-14,0,365418,인권/성평등,이수역 폭행사건,11월 13일 새벽4시 경 이수역의 한 맥주집에서 남자 다섯명이 여성 두 명을 폭행...,30 days,True
369690,2018-09-06,2018-10-06,0,330587,인권/성평등,제 남편의 억울함을 풀어주세요!,제 남편이 어제 재판에서 징역6개월을 받고 그 자리에서 법정구속 되었습니다\n어제 ...,30 days,True


In [42]:
# 일별 투표수
petition_votes = petition.groupby('start')['votes'].sum().reset_index()
petition_votes.head(2)

,start,votes
0,2017-08-19,20703
1,2017-08-20,31710


In [ ]:
# 시각화
(ggplot(petition_votes)
 + aes(x='start', y='votes')
 + geom_point()
 + geom_line(color='blue')
 + labs(x='일자', y='투표수', title='일자별 투표 수')
 + theme(text=element_text(family=font_family),
         figure_size=(12, 6),
         axis_text_x=element_text(rotation=60)))

In [ ]:
# 그래프 확인해서 두드러진 결과를 보여주는 값을 확인한다.
# 백만개의 투표수를 가진 청원이 있다는 것을 알았고 이것을 확인하기로 한다.

## soynly : 명사 추출

In [ ]:
# # 터미널에서 설치
# !pip install soynlp
# !pip show soynlp
# !pip install wordcloud

In [ ]:
# keyword를 추출했다고 가정한다.
import re

p = r'.*(돌봄|육아|초등|보육).*' # raw string / '.*' : 임의 문자가 0개 이상이다.
condition = (petition.title.str.match(p)) | (petition.content.str.match(p, flags=re.MULTILINE)) # flags=re.MULTILINE : 여러 줄에서 패턴 검색 가능
care = petition[condition]
care.head(2)

In [ ]:
# 보고 싶은 청원의 인덱스 설정해주기
sample_idx = 

# 해당 인덱스의 'title' 뽑아주기
sample_title = care.loc[sample_idx, 'title']

# 해당 인덱스의 'content' 뽑아주기
sample_title = care.loc[sample_idx, 'content']

In [ ]:
# 토큰화 하기
from soynlp.tokenizer import RegexTokenizer

# 생성
tokenizer = RegexTokenizer()

# 토큰화하기
tokenized_title = tokenizer.tokenize(sample_title)
tokenized_content = tokenizer.tokenize(sample_content)

In [ ]:
## 텍스트 데이터 전처리하기 ##
# 개행문자 (줄바꿈 문자) 제거 ex) \n
content_text = care.content.str.replace('\\\\n', ' ', regex=True) # '\\\n'인 이유 : 두 개의 \는 하나의 \와 같다. >> '\\\\n' = '\\n' = '\n'

# 숫자 제거
content_text = content_text.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힇a-zA-Z]', ' ', regex=True) # [^] : 뒤에오는 것이 아닌 것

# 토큰화 하기
tokens = content_text.apply(tokenizer.tokenize)

# sample_idx에 맞는 행 추출
tokens[sample_idx]

In [ ]:
# 시각화 - 워드클라우드
!apt -qq -y install fonts-nanum # in google colab
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# 불용어 처리는 수작업으로 하는 것이 나을 것 같다는 판단이 된다.
stopwords = \
['하지만', '그리고', '그런데', '저는','제가',
 '그럼', '이런', '저런', '합니다',
 '많은', '많이', '정말', '너무']

불용어 처리
* 조사, 어미 등 제거 필요
* 한국어 불용어 사전 잘 찾는 것이 좋음
* 메모장을 활용해서 csv 파일을 만들자 ! --> 지속적인 업데이트 필요함.
* LDA를 이용해서 업데이트

In [ ]:
# in Local
def display_WordCloud(data, width=1200, height=500):
    wc = \
    WordCloud(
        font_path='NanumGothic.ttf',
        width=width,
        height=height,
        stopwords=stopwords,
        background_color='white',
        random_state=42
    )
    wc.generate(data)

    plt.figure(figsize=(15, 8))
    plt.imshow(wc)
    plt.axis('off')
    plt.show()

In [ ]:
# 워드클라우드 확인
# 리스트를 string으로 바꿔준 후 위의 함수에 넣어주기
display_WordCloud(' '.join(content_text))

In [ ]:
# 명사 추출하기
from soynlp.noun import LRNounExtractor

%%time

# 추출기 생성
noun_extractor = LRNounExtractor(verbose=True)
# 훈련
noun_extractor.train(content_text)

In [ ]:
# 명사만 추출
nouns = noun_extractor.extract()

# 타입 확인하기
print(type(nouns))

# key 확인
nouns.keys()

In [ ]:
# keys를 리스트로 바꿔서 string으로 바꾸기
nouns_text = ' '.join(list(nouns.keys()))

In [ ]:
# 추출된 명사만을 가지고 워드클라우드 생성하기
display_WordCloud(nouns_text)

## 이진 분류

## 머신러닝